In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [2]:
# pip install music21
# !pip install torch torchvision


In [3]:
import pandas as pd
from ast import literal_eval

notes_df = pd.read_csv ('Dataset/notes.csv')
train_df = pd.read_csv ('Dataset/trainset.csv')
validation_df = pd.read_csv ('Dataset/validationset.csv')

data_train = train_df[['x_tr','future']].to_numpy()
data_validation = validation_df[['x_val','future']].to_numpy()

x_tr_string = data_train[:,0]
y_tr_string = data_train[:,1]
x_tr = []
y_tr = []
for i in x_tr_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_tr = [int(j) for j in i.split()]
    x_tr.append(input_x_tr)

for i in y_tr_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_tr = [int(j) for j in i.split()]
    y_tr.append(input_y_tr)
x_tr = np.array(x_tr)
y_tr = np.array(y_tr)
    
# print(x_tr.shape,y_tr.shape)

x_val_string = data_validation[:,0]
y_val_string = data_validation[:,1]
x_val = []
y_val = []
for i in x_val_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_val = [int(j) for j in i.split()]
    x_val.append(input_x_val)

for i in y_val_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_val = [int(j) for j in i.split()]
    y_val.append(input_y_val)
x_val = np.array(x_val)
y_val = np.array(y_val)

# print(x_val.shape,y_val.shape)

notes_ = notes_df.to_numpy()[:,1]
unique_notes_reverse = dict(enumerate(notes_.flatten(), 1))
unique_notes = {value : key for (key, value) in unique_notes_reverse.items()}


In [4]:
from MusicDataset import *
batch_size = 8
train_set = MusicDataset(x_tr,y_tr)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [5]:
val_set = MusicDataset(x_val,y_val)
validationloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

**DataLoader**

**For Wavenet Model (Ignore for now)**

In [6]:
from Models import Wavenet,LSTM
# Net = Wavenet.Wavenet(len(unique_notes),blocks=4,layers = 2)
Net = LSTM.Sequence(len(unique_notes),32)
Net.to(device)

Sequence(
  (embedding): Embedding(173, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (softmax): Softmax(dim=None)
  (linear3): Linear(in_features=128, out_features=173, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

**Checking the model**

In [7]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

**Use the line below when you want to change learning parameters in between**

In [8]:
import time , sys
baseDir = ''
expDir = os.path.join(baseDir, 'trained_model_cache', time.strftime("%d_%m_%Y_%H_%M"))
lossDir = os.path.join(expDir, 'loss')
os.makedirs(expDir, exist_ok=True)
os.makedirs(lossDir, exist_ok=True)
print(baseDir,expDir)

 trained_model_cache\06_05_2024_23_51


In [9]:
import os

current_directory = os.getcwd()
print("Current directory:", current_directory)


Current directory: C:\Users\Admin\Python\Music_generation\Music-Gen-AI-main


In [10]:
loss_history = []
best_val_acc = 0
training_mode_list = ["Next_Predict_Mode","Future_Predict_Mode"]
training_mode = "Next_Predict_Mode"
# training_mode = "Future_Predict_Mode"
future_preds = 5

In [11]:
# import timeit


# ''' TRAINING THE MODEL '''
# correct_preds = 0
# total_preds = 0
# for epoch in range(20): 
#     running_loss = 0.0
#     start = timeit.default_timer()
#     for i, data in enumerate(trainloader, 0):
#         input , label = data
#         optimizer.zero_grad()
#         if training_mode == training_mode_list[0]:
#             output = Net(input.to(device),input.shape[0])
#             loss = criterion(output.to(device), label[:,0].to(device))
#             loss.backward()
#             optimizer.step()
#             total_preds += input.shape[0]
#             correct_preds += torch.sum(torch.argmax(output,1) == label[:,0].to(device))

#             running_loss += loss.item()
#         elif training_mode == training_mode_list[1]:
#             cumm_output = torch.zeros(0,len(unique_notes)).to(device)
#             cumm_label  = np.array([],dtype=float)
#             for k in range(future_preds):
#                 output = Net(input.to(device),input.shape[0])
#                 cumm_output = torch.cat((cumm_output,output))
#                 cumm_label = np.concatenate((cumm_label,label[:,k]))
#                 next_preds = np.argmax((output.cpu().detach().numpy(),axis=1)).long()
#                 input = (input.cpu().detach().numpy()).long()
#                 input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
#                                                        for ind,j in enumerate(input)])[:,1:])  
#             loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()
        
#         if (i % 300 == 299  ):
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 300))
#             loss_history.append(running_loss/300)
#             running_loss = 0.0
#     stop = timeit.default_timer()
#     print(f"Training Time for Epoch :{epoch} \n {stop-start} seconds\n")
#     train_acc =  float(correct_preds)/float(total_preds) *100 
#     trainreport ="Training Accuracy : \n correct predictions  : {} \n total predictions : {} \n Training Accuracy : {} \n ------------------------\n".format(correct_preds,total_preds,train_acc)
#     print(trainreport)     
#     correct_preds = 0
#     total_preds = 0
#     val_loss = 0
#     start = timeit.default_timer()
#     for i, data in enumerate(validationloader, 0):
#         input , label = data
#         if training_mode == training_mode_list[0]:
#             output = Net(input.to(device),input.shape[0])
#             loss = criterion(output.to(device), label[:,0].to(device))
#             total_preds += input.shape[0]
#             correct_preds += torch.sum(torch.argmax(output,1) == label[:,0].to(device))
#             val_loss += loss.item()
            
#         elif training_mode == training_mode_list[1]:
#             cumm_output = torch.zeros(0,len(unique_notes)).to(device)
#             cumm_label  = np.array([],dtype=float)
#             for k in range(future_preds):
#                 output = Net(input.to(device),input.shape[0])
#                 cumm_output = torch.cat((cumm_output,output))
#                 cumm_label = np.concatenate((cumm_label,label[:,k]))
#                 next_preds = np.argmax((output.cpu().detach().numpy(),axis=1)).long()
#                 input = (input.cpu().detach().numpy()).long()
#                 input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
#                                                        for ind,j in enumerate(input)])[:,1:])  
#             loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))

#             val_loss += loss.item()
#     stop = timeit.default_timer()
#     print(f"Validation Time for Epoch :{epoch} \n {stop-start} seconds\n")
#     val_acc = float(correct_preds)/float(total_preds) *100
#     val_loss = float(val_loss)/float(total_preds)
#     valreport ="Validation Accuracy : \n correct predictions  : {} \n total predictions : {} \n Val-Loss : {} \n Val-Accuracy :{}----------------------------------\n".format(correct_preds,total_preds,val_loss,val_acc )
#     print(valreport)
    
#     if(val_acc > best_val_acc):
#         print('Updating the model')
#         best_val_acc = val_acc
#         checkpoint = {
#             "epoch": epoch,
#             "model_state_dict": Net.state_dict(),
#             "optimizer_state_dict": optimizer.state_dict(),
#             "train_acc": train_acc,
#             "valid_acc": val_acc
#         }
#         torch.save(checkpoint, os.path.join(expDir, 'checkpoint.tar'))
#         torch.save(Net, os.path.join(expDir, 'model.pth'))
    
#     correct_preds = 0
#     total_preds = 0


# print('Finished Training')

In [12]:
# ################Early Stopping

# import timeit
# import numpy as np
# import torch
# import os

# ''' TRAINING THE MODEL '''
# correct_preds = 0
# total_preds = 0
# best_val_acc = 0
# patience = 5  # Number of epochs to wait before stopping if no improvement
# count = 0  # Counter for patience

# for epoch in range(20):
#     running_loss = 0.0
#     start = timeit.default_timer()
    
#     for i, data in enumerate(trainloader, 0):
#         input, label = data
#         optimizer.zero_grad()
        
#         if training_mode == training_mode_list[0]:
#             output = Net(input.to(device), input.shape[0])
#             loss = criterion(output.to(device), label[:,0].to(device).long())
#             loss.backward()
#             optimizer.step()
#             total_preds += input.shape[0]
#             correct_preds += torch.sum(torch.argmax(output, 1) == label[:,0].to(device))
#             running_loss += loss.item()
            
#         elif training_mode == training_mode_list[1]:
#             cumm_output = torch.zeros(0, len(unique_notes)).to(device)
#             cumm_label = np.array([], dtype=float)
#             for k in range(future_preds):
#                 output = Net(input.to(device), input.shape[0])
#                 cumm_output = torch.cat((cumm_output, output))
#                 cumm_label = np.concatenate((cumm_label, label[:, k]))
#                 next_preds = torch.argmax(output.cpu().detach().numpy(), axis=1).long()
#                 input = input.cpu().detach().numpy().astype(np.int64)
#                 input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
#                                                    for ind, j in enumerate(input)])[:, 1:]).long()
#             loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()
        
#         if (i % 300 == 299):
#             print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 300))
#             loss_history.append(running_loss / 300)
#             running_loss = 0.0
            
#     stop = timeit.default_timer()
#     print(f"Training Time for Epoch :{epoch} \n {stop-start} seconds\n")
#     train_acc = float(correct_preds) / float(total_preds) * 100 
#     trainreport = "Training Accuracy : \n correct predictions  : {} \n total predictions : {} \n Training Accuracy : {} \n ------------------------\n".format(correct_preds, total_preds, train_acc)
#     print(trainreport)     
#     correct_preds = 0
#     total_preds = 0
    
#     val_loss = 0
#     start = timeit.default_timer()
#     for i, data in enumerate(validationloader, 0):
#         input , label = data
#         if training_mode == training_mode_list[0]:
#             output = Net(input.to(device), input.shape[0])
#             loss = criterion(output.to(device), label[:,0].to(device).long())
#             total_preds += input.shape[0]
#             correct_preds += torch.sum(torch.argmax(output, 1) == label[:,0].to(device))
#             val_loss += loss.item()
#         elif training_mode == training_mode_list[1]:
#             cumm_output = torch.zeros(0, len(unique_notes)).to(device)
#             cumm_label  = np.array([], dtype=float)
#             for k in range(future_preds):
#                 output = Net(input.to(device), input.shape[0])
#                 cumm_output = torch.cat((cumm_output, output))
#                 cumm_label = np.concatenate((cumm_label, label[:, k]))
#                 next_preds = torch.argmax(output.cpu().detach().numpy(), axis=1).long()
#                 input = input.cpu().detach().numpy().astype(np.int64)
#                 input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
#                                                    for ind, j in enumerate(input)])[:, 1:]).long()
#             loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))
#             val_loss += loss.item()
            
#     stop = timeit.default_timer()
#     print(f"Validation Time for Epoch :{epoch} \n {stop-start} seconds\n")
#     val_acc = float(correct_preds) / float(total_preds) * 100
#     val_loss = float(val_loss) / float(total_preds)
#     valreport = "Validation Accuracy : \n correct predictions  : {} \n total predictions : {} \n Val-Loss : {} \n Val-Accuracy : {}----------------------------------\n".format(correct_preds, total_preds, val_loss, val_acc)
#     print(valreport)
    
#     if val_acc > best_val_acc:
#         print('Updating the model')
#         best_val_acc = val_acc
#         count = 0  # Reset patience counter
#         checkpoint = {
#             "epoch": epoch,
#             "model_state_dict": Net.state_dict(),
#             "optimizer_state_dict": optimizer.state_dict(),
#             "train_acc": train_acc,
#             "valid_acc": val_acc
#         }
#         torch.save(checkpoint, os.path.join(expDir, 'checkpoint.tar'))
#         torch.save(Net, os.path.join(expDir, 'model.pth'))
#     else:
#         count += 1
#         if count >= patience:
#             print("Early stopping triggered, no improvement in validation accuracy.")
#             break
    
#     correct_preds = 0
#     total_preds = 0

# print('Finished Training')



[1,   300] loss: 4.829
[1,   600] loss: 4.688
[1,   900] loss: 4.632
[1,  1200] loss: 4.613
[1,  1500] loss: 4.606
[1,  1800] loss: 4.555
[1,  2100] loss: 4.524
[1,  2400] loss: 4.550


KeyboardInterrupt: 

In [13]:
import timeit
import numpy as np
import torch
import os

''' TRAINING THE MODEL '''
correct_preds = 0
total_preds = 0
for epoch in range(90): 
    running_loss = 0.0
    start = timeit.default_timer()
    for i, data in enumerate(trainloader, 0):
        input, label = data
        optimizer.zero_grad()
        if training_mode == training_mode_list[0]:
            output = Net(input.to(device), input.shape[0])
            loss = criterion(output.to(device), label[:,0].to(device).long())
            loss.backward()
            optimizer.step()
            total_preds += input.shape[0]
            correct_preds += torch.sum(torch.argmax(output, 1) == label[:,0].to(device))
            running_loss += loss.item()
        elif training_mode == training_mode_list[1]:
            cumm_output = torch.zeros(0, len(unique_notes)).to(device)
            cumm_label = np.array([], dtype=float)
            for k in range(future_preds):
                output = Net(input.to(device), input.shape[0])
                cumm_output = torch.cat((cumm_output, output))
                cumm_label = np.concatenate((cumm_label, label[:, k]))
                next_preds = torch.argmax(output.cpu().detach().numpy(), axis=1).long()
                input = input.cpu().detach().numpy().astype(np.int64)
                input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
                                                   for ind, j in enumerate(input)])[:, 1:]).long()
            loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        if (i % 300 == 299):
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 300))
            loss_history.append(running_loss / 300)
            running_loss = 0.0
    stop = timeit.default_timer()
    print(f"Training Time for Epoch :{epoch} \n {stop-start} seconds\n")
    train_acc = float(correct_preds) / float(total_preds) * 100 
    trainreport = "Training Accuracy : \n correct predictions  : {} \n total predictions : {} \n Training Accuracy : {} \n ------------------------\n".format(correct_preds, total_preds, train_acc)
    print(trainreport)     
    correct_preds = 0
    total_preds = 0
    val_loss = 0
    start = timeit.default_timer()
    for i, data in enumerate(validationloader, 0):
        input , label = data
        if training_mode == training_mode_list[0]:
            output = Net(input.to(device), input.shape[0])
            loss = criterion(output.to(device), label[:,0].to(device).long())
            total_preds += input.shape[0]
            correct_preds += torch.sum(torch.argmax(output, 1) == label[:,0].to(device))
            val_loss += loss.item()
        elif training_mode == training_mode_list[1]:
            cumm_output = torch.zeros(0, len(unique_notes)).to(device)
            cumm_label  = np.array([], dtype=float)
            for k in range(future_preds):
                output = Net(input.to(device), input.shape[0])
                cumm_output = torch.cat((cumm_output, output))
                cumm_label = np.concatenate((cumm_label, label[:, k]))
                next_preds = torch.argmax(output.cpu().detach().numpy(), axis=1).long()
                input = input.cpu().detach().numpy().astype(np.int64)
                input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
                                                   for ind, j in enumerate(input)])[:, 1:]).long()
            loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))
            val_loss += loss.item()
    stop = timeit.default_timer()
    print(f"Validation Time for Epoch :{epoch} \n {stop-start} seconds\n")
    val_acc = float(correct_preds) / float(total_preds) * 100
    val_loss = float(val_loss) / float(total_preds)
    valreport = "Validation Accuracy : \n correct predictions  : {} \n total predictions : {} \n Val-Loss : {} \n Val-Accuracy : {}----------------------------------\n".format(correct_preds, total_preds, val_loss, val_acc)
    print(valreport)
    
    if val_acc > best_val_acc:
        print('Updating the model')
        best_val_acc = val_acc
        checkpoint = {
            "epoch": epoch,
            "model_state_dict": Net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_acc": train_acc,
            "valid_acc": val_acc
        }
        torch.save(checkpoint, os.path.join(expDir, 'checkpoint.tar'))
        torch.save(Net, os.path.join(expDir, 'model.pth'))
    
    correct_preds = 0
    total_preds = 0

print('Finished Training')


[1,   300] loss: 4.490
[1,   600] loss: 4.446
[1,   900] loss: 4.420
[1,  1200] loss: 4.407
[1,  1500] loss: 4.391
[1,  1800] loss: 4.353
[1,  2100] loss: 4.329
[1,  2400] loss: 4.322
[1,  2700] loss: 4.339
[1,  3000] loss: 4.257
[1,  3300] loss: 4.268
[1,  3600] loss: 4.273
[1,  3900] loss: 4.229
[1,  4200] loss: 4.238
[1,  4500] loss: 4.212
[1,  4800] loss: 4.207
[1,  5100] loss: 4.138
[1,  5400] loss: 4.134
[1,  5700] loss: 4.144
[1,  6000] loss: 4.105
[1,  6300] loss: 4.136
[1,  6600] loss: 4.135
[1,  6900] loss: 4.104
Training Time for Epoch :0 
 417.66239559999667 seconds

Training Accuracy : 
 correct predictions  : 3245 
 total predictions : 56095 
 Training Accuracy : 5.7848293074249035 
 ------------------------

Validation Time for Epoch :0 
 36.977737199980766 seconds

Validation Accuracy : 
 correct predictions  : 1278 
 total predictions : 16028 
 Val-Loss : 0.5097800918222812 
 Val-Accuracy : 7.9735462939855255----------------------------------

Updating the model
[2,   

[9,  3900] loss: 2.425
[9,  4200] loss: 2.358
[9,  4500] loss: 2.392
[9,  4800] loss: 2.435
[9,  5100] loss: 2.403
[9,  5400] loss: 2.391
[9,  5700] loss: 2.407
[9,  6000] loss: 2.417
[9,  6300] loss: 2.431
[9,  6600] loss: 2.403
[9,  6900] loss: 2.410
Training Time for Epoch :8 
 409.00117989996215 seconds

Training Accuracy : 
 correct predictions  : 20169 
 total predictions : 56095 
 Training Accuracy : 35.95507621000089 
 ------------------------

Validation Time for Epoch :8 
 36.37616059998982 seconds

Validation Accuracy : 
 correct predictions  : 4404 
 total predictions : 16028 
 Val-Loss : 0.4104110546092128 
 Val-Accuracy : 27.476915398053407----------------------------------

Updating the model
[10,   300] loss: 2.386
[10,   600] loss: 2.370
[10,   900] loss: 2.306
[10,  1200] loss: 2.360
[10,  1500] loss: 2.323
[10,  1800] loss: 2.401
[10,  2100] loss: 2.312
[10,  2400] loss: 2.298
[10,  2700] loss: 2.287
[10,  3000] loss: 2.325
[10,  3300] loss: 2.359
[10,  3600] loss: 2

[17,  4800] loss: 2.011
[17,  5100] loss: 1.913
[17,  5400] loss: 1.955
[17,  5700] loss: 2.003
[17,  6000] loss: 1.981
[17,  6300] loss: 1.960
[17,  6600] loss: 2.046
[17,  6900] loss: 1.996
Training Time for Epoch :16 
 407.64070170000196 seconds

Training Accuracy : 
 correct predictions  : 26713 
 total predictions : 56095 
 Training Accuracy : 47.621000089134505 
 ------------------------

Validation Time for Epoch :16 
 36.63179159996798 seconds

Validation Accuracy : 
 correct predictions  : 5360 
 total predictions : 16028 
 Val-Loss : 0.4308325421127382 
 Val-Accuracy : 33.441477414524584----------------------------------

Updating the model
[18,   300] loss: 2.031
[18,   600] loss: 1.959
[18,   900] loss: 1.957
[18,  1200] loss: 1.945
[18,  1500] loss: 1.947
[18,  1800] loss: 1.992
[18,  2100] loss: 1.938
[18,  2400] loss: 1.966
[18,  2700] loss: 1.927
[18,  3000] loss: 1.982
[18,  3300] loss: 1.980
[18,  3600] loss: 1.959
[18,  3900] loss: 1.981
[18,  4200] loss: 1.940
[18, 

[25,  6000] loss: 1.871
[25,  6300] loss: 1.895
[25,  6600] loss: 1.965
[25,  6900] loss: 1.901
Training Time for Epoch :24 
 406.7084968999843 seconds

Training Accuracy : 
 correct predictions  : 28433 
 total predictions : 56095 
 Training Accuracy : 50.687227025581606 
 ------------------------

Validation Time for Epoch :24 
 37.00268440000946 seconds

Validation Accuracy : 
 correct predictions  : 5673 
 total predictions : 16028 
 Val-Loss : 0.43302293522699475 
 Val-Accuracy : 35.394309957574244----------------------------------

[26,   300] loss: 1.898
[26,   600] loss: 1.907
[26,   900] loss: 1.874
[26,  1200] loss: 1.865
[26,  1500] loss: 1.878
[26,  1800] loss: 1.916
[26,  2100] loss: 1.863
[26,  2400] loss: 1.916
[26,  2700] loss: 1.867
[26,  3000] loss: 1.904
[26,  3300] loss: 1.873
[26,  3600] loss: 1.910
[26,  3900] loss: 1.897
[26,  4200] loss: 1.897
[26,  4500] loss: 1.840
[26,  4800] loss: 1.894
[26,  5100] loss: 1.854
[26,  5400] loss: 1.884
[26,  5700] loss: 1.887


Validation Time for Epoch :32 
 36.89809540001443 seconds

Validation Accuracy : 
 correct predictions  : 5620 
 total predictions : 16028 
 Val-Loss : 0.43392926931291975 
 Val-Accuracy : 35.063638632393314----------------------------------

[34,   300] loss: 1.917
[34,   600] loss: 1.907
[34,   900] loss: 1.850
[34,  1200] loss: 1.864
[34,  1500] loss: 1.846
[34,  1800] loss: 1.899
[34,  2100] loss: 1.822
[34,  2400] loss: 1.887
[34,  2700] loss: 1.884
[34,  3000] loss: 1.888
[34,  3300] loss: 1.906
[34,  3600] loss: 1.926
[34,  3900] loss: 1.870
[34,  4200] loss: 1.840
[34,  4500] loss: 1.832
[34,  4800] loss: 1.920
[34,  5100] loss: 1.863
[34,  5400] loss: 1.888
[34,  5700] loss: 1.957
[34,  6000] loss: 1.914
[34,  6300] loss: 1.964
[34,  6600] loss: 1.954
[34,  6900] loss: 1.965
Training Time for Epoch :33 
 405.8104179999791 seconds

Training Accuracy : 
 correct predictions  : 28631 
 total predictions : 56095 
 Training Accuracy : 51.04019966128889 
 ------------------------

V

...

[50,  3000] loss: 1.932
[50,  3300] loss: 1.934
[50,  3600] loss: 1.956
[50,  3900] loss: 1.963
[50,  4200] loss: 1.931
[50,  4500] loss: 1.931
[50,  4800] loss: 2.008
[50,  5100] loss: 1.929
[50,  5400] loss: 1.906
[50,  5700] loss: 1.983
[50,  6000] loss: 1.999
[50,  6300] loss: 1.974
[50,  6600] loss: 1.950
[50,  6900] loss: 1.960
Training Time for Epoch :49 
 413.64280290005263 seconds

Training Accuracy : 
 correct predictions  : 28346 
 total predictions : 56095 
 Training Accuracy : 50.532132988679926 
 ------------------------

Validation Time for Epoch :49 
 38.25491509999847 seconds

Validation Accuracy : 
 correct predictions  : 5513 
 total predictions : 16028 
 Val-Loss : 0.4397967115234252 
 Val-Accuracy : 34.39605690042426----------------------------------

[51,   300] loss: 2.007
[51,   600] loss: 2.025
[51,   900] loss: 1.921
[51,  1200] loss: 2.025
[51,  1500] loss: 1.936
[51,  1800] loss: 2.010
[51,  2100] loss: 1.986
[51,  2400] loss: 1.902
[51,  2700] loss: 1.899
[

[58,  5700] loss: 2.035
[58,  6000] loss: 1.960
[58,  6300] loss: 1.968
[58,  6600] loss: 1.992
[58,  6900] loss: 1.932
Training Time for Epoch :57 
 407.4811972999596 seconds

Training Accuracy : 
 correct predictions  : 28272 
 total predictions : 56095 
 Training Accuracy : 50.40021392280952 
 ------------------------

Validation Time for Epoch :57 
 38.297836199984886 seconds

Validation Accuracy : 
 correct predictions  : 5572 
 total predictions : 16028 
 Val-Loss : 0.4419655541001278 
 Val-Accuracy : 34.76416271524832----------------------------------

[59,   300] loss: 1.988
[59,   600] loss: 2.002
[59,   900] loss: 1.946
[59,  1200] loss: 2.032
[59,  1500] loss: 1.956
[59,  1800] loss: 2.001
[59,  2100] loss: 1.946
[59,  2400] loss: 1.990
[59,  2700] loss: 2.015
[59,  3000] loss: 1.970
[59,  3300] loss: 1.984
[59,  3600] loss: 1.995
[59,  3900] loss: 1.993
[59,  4200] loss: 1.973
[59,  4500] loss: 1.917
[59,  4800] loss: 1.963
[59,  5100] loss: 1.928
[59,  5400] loss: 1.960
[5

Validation Time for Epoch :65 
 37.85820410004817 seconds

Validation Accuracy : 
 correct predictions  : 5532 
 total predictions : 16028 
 Val-Loss : 0.4352868555905309 
 Val-Accuracy : 34.51459945096082----------------------------------

[67,   300] loss: 2.035
[67,   600] loss: 2.018
[67,   900] loss: 1.979
[67,  1200] loss: 2.060
[67,  1500] loss: 1.975
[67,  1800] loss: 2.069
[67,  2100] loss: 1.993
[67,  2400] loss: 1.992
[67,  2700] loss: 1.972
[67,  3000] loss: 1.981
[67,  3300] loss: 1.979
[67,  3600] loss: 1.981
[67,  3900] loss: 2.069
[67,  4200] loss: 1.994
[67,  4500] loss: 1.967
[67,  4800] loss: 2.009
[67,  5100] loss: 1.954
[67,  5400] loss: 1.965
[67,  5700] loss: 2.047
[67,  6000] loss: 2.016
[67,  6300] loss: 2.060
[67,  6600] loss: 2.072
[67,  6900] loss: 2.046
Training Time for Epoch :66 
 404.69114999996964 seconds

Training Accuracy : 
 correct predictions  : 27707 
 total predictions : 56095 
 Training Accuracy : 49.39299402798824 
 ------------------------

Va

[75,   300] loss: 2.057
[75,   600] loss: 2.063
[75,   900] loss: 2.038
[75,  1200] loss: 2.061
[75,  1500] loss: 2.047
[75,  1800] loss: 2.133
[75,  2100] loss: 2.001
[75,  2400] loss: 2.025
[75,  2700] loss: 2.013
[75,  3000] loss: 2.052
[75,  3300] loss: 2.026
[75,  3600] loss: 2.042
[75,  3900] loss: 2.086
[75,  4200] loss: 2.017
[75,  4500] loss: 2.068
[75,  4800] loss: 2.127
[75,  5100] loss: 1.969
[75,  5400] loss: 2.005
[75,  5700] loss: 2.103
[75,  6000] loss: 2.038
[75,  6300] loss: 2.125
[75,  6600] loss: 2.098
[75,  6900] loss: 2.014
Training Time for Epoch :74 
 522.3875023999717 seconds

Training Accuracy : 
 correct predictions  : 27485 
 total predictions : 56095 
 Training Accuracy : 48.99723683037704 
 ------------------------

Validation Time for Epoch :74 
 72.44725500000641 seconds

Validation Accuracy : 
 correct predictions  : 5542 
 total predictions : 16028 
 Val-Loss : 0.4357050626332069 
 Val-Accuracy : 34.57699026703269----------------------------------

[76

[83,  3000] loss: 2.085
[83,  3300] loss: 2.135
[83,  3600] loss: 2.114
[83,  3900] loss: 2.101
[83,  4200] loss: 2.030
[83,  4500] loss: 2.084
[83,  4800] loss: 2.098
[83,  5100] loss: 2.055
[83,  5400] loss: 2.072
[83,  5700] loss: 2.099
[83,  6000] loss: 2.093
[83,  6300] loss: 2.163
[83,  6600] loss: 2.116
[83,  6900] loss: 2.047
Training Time for Epoch :82 
 574.2209671000019 seconds

Training Accuracy : 
 correct predictions  : 27204 
 total predictions : 56095 
 Training Accuracy : 48.49630091808539 
 ------------------------

Validation Time for Epoch :82 
 43.921662999957334 seconds

Validation Accuracy : 
 correct predictions  : 5412 
 total predictions : 16028 
 Val-Loss : 0.432327088346974 
 Val-Accuracy : 33.76590965809832----------------------------------

[84,   300] loss: 2.126
[84,   600] loss: 2.129
[84,   900] loss: 2.056
[84,  1200] loss: 2.073
[84,  1500] loss: 2.058
[84,  1800] loss: 2.082
[84,  2100] loss: 2.074
[84,  2400] loss: 2.082
[84,  2700] loss: 2.077
[84

In [14]:
# plt.figure()
# plt.plot(loss_history)
# # plt.savefig(lossDir + '/loss_history.png')

In [15]:
# path='/content/drive/MyDrive/schubert/sine_model.pth'
# path='/content/drive/MyDrive/schubert/lstmmodel1.pth'
path='schubert/sine_wavenet1.pth'
torch.save(Net.state_dict(), path)

In [16]:
##############Error

# # path='/content/drive/MyDrive/schubert/sine_model.pth'
# # path='/content/drive/MyDrive/schubert/lstmmodel1.pth'
# checkpoint_path='trained_model_cache/06_05_2024_17_21/checkpoint.tar'
# Net = LSTM.Sequence(len(unique_notes),32);
# checkpoint = torch.load(checkpoint_path)
# Net.load_state_dict(checkpoint["model_state_dict"])
# print(checkpoint["epoch"])
# # Net = Wavenet(len(unique_x),blocks=4,layers = 2)
# # Net.load_state_dict(torch.load(path))

# correct_preds = 0
# total_preds = 0
# with torch.no_grad():
#   for i, data in enumerate(validationloader, 0):
#     input , label = data
#     output = Net(input,input.shape[0])
#     total_preds += input.shape[0]
#     correct_preds += torch.sum(torch.argmax(output,1) == label)
    

# report ="correct predictions  : {} \n total predictions : {} \n Accuracy : {}".format(correct_preds,total_preds, correct_preds/total_preds *100)
# print(report)


In [17]:
correct_preds = 0
total_preds = 0

with torch.no_grad():
    for i, data in enumerate(validationloader, 0):
        input, label = data
        output = Net(input, input.shape[0])
        total_preds += input.shape[0]
        _, predicted = torch.max(output, 1)
        correct_preds += (predicted == label[:, 0]).sum().item()  # Adjust the index to match your label structure

report = "Correct predictions: {} \nTotal predictions: {} \nAccuracy: {:.2f}%".format(correct_preds, total_preds, (correct_preds / total_preds) * 100)
print(report)


Correct predictions: 5455 
Total predictions: 16028 
Accuracy: 34.03%


**Testing the Model by training sinosoids**



*   Uncomment the code chunk below to test o sinosoids 
*   dont run above code if training for sinosoids. Variable names are mostly same



In [18]:
time = np.linspace(1,10,10000)
# print(time)
y = np.sin(20*time)
# plt.figure();
# plt.plot(y)
func = np.round(y,decimals=2)
# print(y[:100])
no_of_timesteps = 32
x = []
y = []


for i in range(0, len(func) - no_of_timesteps-1, 1):
    
    #preparing input and output sequences
    input_ = func[i:i + no_of_timesteps]
    output = func[i+no_of_timesteps+1]
    
    x.append(input_)
    y.append(output)
        
x=np.array(x)
y=np.array(y)

unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)

unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_seq=np.array([x_note_to_int[i] for i in y])
